<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/weinstein_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Stock Screening Inspired by Stan Weinstein

This notebook features stock screening methods inspired by Stan Weinstein's book, *Secrets for Profiting in Bull and Bear Markets*. The strategies and criteria used in this analysis are based on the principles outlined in Weinstein's work, aiming to identify potential investment opportunities in both bull and bear markets.


### Install and Setup (this section will be executed automatically)

#### Install Required Packages

In [28]:
%pip install "git+https://github.com/yorkjong/vistock.git"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git to /tmp/pip-req-build-mbhs72ia
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-mbhs72ia
  Resolved https://github.com/yorkjong/vistock.git to commit 7b831c9545c49db2b4b579efc1af2c915915fbf2
  Preparing metadata (setup.py) ... done
--2024-09-08 18:59:04--  https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0
Resolving share.cole.tw (share.cole.tw)... 180.218.0.193
Connecting to share.cole.tw (share.cole.tw)|180.218.0.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7110560 (6.8M) [application/octet-stream]
Saving to: ‘NotoSansTC-Regular.ttf’

NotoSansTC-Regular. 100%[===================>]   6.78M  3.31MB/s    in 2.0s    

2024-09-08 18:59:08 (3.31 MB/s) - ‘NotoSansTC-Regular.ttf’ saved [7110560/7110560]



In [29]:
# @title Install Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

#### Setup and Configuration

In [30]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [31]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [32]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    dir='weinstein'
)

In [33]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [34]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return list(df['Name'])
    elif 'Ticker' in df.columns:
        return list(df['Ticker'])
    return []

def sorting_tickers():
    return list(tickers_from_df(df_top_f1))

def increasing_rs_tickers():
    return list(tickers_from_df(df_top_f2))

def rs_breakout_tickers():
    return list(tickers_from_df(df_top_f3))

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

In [35]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

In [36]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [37]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The Source of Stocks to Analyze)
- **Description**: Stocks can be sourced from various exchanges or indices.
- **Common Abbreviations**:
  - **Taiwan Markets**:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所)
    - `TPEX`: Taipei Exchange (上櫃交易所)
    - `ESB`: Emerging Stock Board (興櫃交易所)
    - Combine with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - **America Markets**:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index (費半指數)
    - Combine with `+` (e.g., `SPX+DJIA+NDX+SOX`)

#### Period (Historical Data Time Range)
- **Description**: The duration for fetching historical data.
- **Example**: `2y` for 2 years

#### Interval (Historical Data Frequency)
- **Description**: The frequency of historical data points.
- **Common Intervals**:
  - `'1d'`: Daily data
  - `'1wk'`: Weekly data
  - `'1mo'`: Monthly data

#### MA (Moving Average)
- **SMA**: Simple Moving Average
- **EMA**: Exponential Moving Average

#### RS (Relative Strength)
- **Description**: Measures a stock's performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the performance of the benchmark index or market.
  - A positive value indicates outperformance relative to the benchmark.
  - A negative value indicates underperformance relative to the benchmark.

#### EPS RS (EPS Relative Strength)
- **Description**: Measures a stock's earnings per share (EPS) performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the EPS performance of the benchmark index.
  - A positive value indicates that the stock's EPS growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index EPS Calculation**: Weighted according to component stocks' share counts.

#### Rev RS (Operating Revenue Relative Strength)
- **Description**: Measures a stock's operating revenue performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the revenue performance of the benchmark index.
  - A positive value indicates that the stock's revenue growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index Revenue Calculation**: Weighted according to component stocks' market capitalization.

#### TTM EPS (Trailing Twelve Months EPS)
- **Description**: The earnings per share over the trailing twelve months, which gives a more recent view of the company's profitability.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM EPS indicates better profitability over the past year.
  - Used to compare with previous periods or with other companies.

#### TTM RPS (Trailing Twelve Months Revenue Per Share)
- **Description**: The revenue per share over the trailing twelve months, showing how much revenue a company generates per share.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM RPS indicates better revenue generation capability per share over the past year.
  - Useful for comparing revenue performance with previous periods or with other companies.

#### TTM PE (Trailing Twelve Months PE)
- **Description**: The price-to-earnings ratio calculated using the earnings from the trailing twelve months. It provides a measure of how much investors are willing to pay per dollar of earnings.
- **Unit**: Ratio
- **Interpretation**:
  - A higher TTM PE indicates that investors expect higher future growth and are willing to pay more for the stock.
  - A lower TTM PE might suggest undervaluation or lower expected growth.


##### References:
- [Mansfield Relative Strength (Original Version) by stageanalysis — Indicator by Stage_Analysis — TradingView](https://www.tradingview.com/script/NzUBDDtb-Mansfield-Relative-Strength-Original-Version-by-stageanalysis/)
- [Mansfield Relative Strength | TrendSpider Store](https://trendspider.com/trading-tools-store/indicators/mansfield-relative-strength/)
- [Mansfield Relative Strength | ChartMill.com](https://www.chartmill.com/documentation/technical-analysis/indicators/35-Mansfield-Relative-Strength)
- [How to create the Mansfield Relative Performance Indicator - Stage Analysis](https://www.stageanalysis.net/blog/4266/how-to-create-the-mansfield-relative-performance-indicator)
  - [Stan Weinstein's Stage Analysis | Page 49 | Trade2Win Forums • UK Financial Trading Community](https://www.trade2win.com/threads/stan-weinsteins-stage-analysis.134944/page-49#post-2137398)


## Execute Actions Step by Step


In [18]:
# @title Step 1. Pick a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[4].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames), reverse=True)
all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
all_periods = sorted(set(fn.split('_')[2] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames to set dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=all_types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

period_dropdown = widgets.Dropdown(
    options=all_periods,
    description='Period:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files), reverse=True)
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update Type dropdown
    available_types = sorted(set(fn.split('_')[1] for fn in date_filtered_files), reverse=True)
    type_dropdown.options = available_types
    if type_dropdown.value not in available_types:
        type_dropdown.value = available_types[0] if available_types else None

    # Update Period dropdown
    available_periods = sorted(set(fn.split('_')[2] for fn in date_filtered_files), reverse=True)
    period_dropdown.options = available_periods
    if period_dropdown.value not in available_periods:
        period_dropdown.value = available_periods[0] if available_periods else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            type_dropdown.value == fn.split('_')[1] and
            period_dropdown.value == fn.split('_')[2])
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')
period_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, period_dropdown, file_dropdown)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='470px'), options=('20240908',), value='20240908')

Dropdown(description='Source:', layout=Layout(width='470px'), options=('TWSE+TPEX', 'SPX+DJIA+NDX+RUI+SOX'), v…

Dropdown(description='Type:', layout=Layout(width='470px'), options=('stocks',), value='stocks')

Dropdown(description='Period:', layout=Layout(width='470px'), options=('2y',), value='2y')

Dropdown(description='File:', layout=Layout(width='470px'), options=(), value=None)

### Step 2. Filter Stocks

In [24]:
# @title Filter 1. Sorting {"run":"auto"}
cond0 = "(df['RS (%)'] > 0)" # @param ["(df['RS (%)'] > 0)"]
cond1 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond2 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond3 = "& (df['Rev RS (%)'] > 0)" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond4 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond5 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
sorted_column = "RS (%)" # @param ["RS (%)","1 Week Ago","1 Month Ago","3 Months Ago","6 Months Ago","Price","MA10","MA30","Volume / VMA10","EPS RS (%)", "Rev RS (%)"]
ascending = False # @param {"type":"boolean"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 6
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, ma, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nMA: {ma}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

Source: SPX+DJIA+NDX+RUI+SOX
Type: stocks
Period: 2y
MA: SMA
Date: 20240908


,Ticker,Sector,Industry,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,RS Rank (%),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
2,SN,Consumer Cyclical,"Furnishings, Fixtures & Appliances",44.94,42.61,25.56,37.26,12.05,15.92,99.80,97.34,82.72,71.73,1.50,5.65,1.75,9.24,34.176,54.11
7,VIRT,Financial Services,Capital Markets,36.19,29.60,25.95,3.31,-12.26,-10.25,99.30,31.44,28.05,23.37,0.79,37.72,1.99,7.27,20.318,15.64
11,APP,Technology,Software - Application,29.85,37.90,12.60,55.65,63.12,42.54,98.90,88.19,83.11,76.58,0.99,44.49,2.34,5.73,11.726,36.14
14,PLTR,Technology,Software—Infrastructure,28.88,29.34,11.99,4.18,41.91,52.19,98.60,30.16,29.15,25.12,0.80,27.62,0.17,5.93,1.127,178.41
16,HWM,Industrials,Aerospace & Defense,28.42,31.17,34.17,34.94,18.66,8.83,98.40,94.21,88.30,78.09,1.16,9.33,2.25,2.53,17.300,40.64
20,K,Consumer Defensive,Packaged Foods,27.34,23.18,2.49,-5.44,-18.90,-20.26,98.00,80.19,68.79,61.36,0.69,46.17,2.51,8.31,37.479,31.91
22,GME,Consumer Cyclical,Specialty Retail,27.02,19.38,13.17,22.52,-29.77,-28.03,97.80,22.39,23.43,19.32,1.37,394.29,0.08,32.57,16.099,299.00
34,NU,Financial Services,Banks - Regional,21.53,28.34,2.23,18.10,31.34,23.54,96.65,14.34,13.26,12.12,1.13,22.91,0.31,11.56,1.029,44.19
43,JEF,Financial Services,Capital Markets,19.73,23.03,20.28,8.05,3.09,-2.71,95.70,57.79,55.79,48.21,1.05,36.22,1.84,2.91,25.990,30.50
57,HRB,Consumer Cyclical,Personal Services,18.16,14.46,10.63,2.61,6.03,14.05,94.31,63.55,58.90,52.46,1.30,160.42,4.14,14.24,25.437,15.20


SN, VIRT, APP, PLTR, HWM, K, GME, NU, JEF, HRB


In [38]:
# @title Filter 2. Increasing RS > 0 {"run":"auto"}
cond0 = "(df['RS (%)'] > 0)" # @param ["(df['RS (%)'] > 0)"]
cond1 = "& (df['RS (%)'] > df['1 Week Ago'])" # @param ["& (df['RS (%)'] > df['1 Week Ago'])", ""]
cond2 = "& (df['1 Week Ago'] > df['1 Month Ago'])" # @param ["& (df['1 Week Ago'] > df['1 Month Ago'])", "& (df['RS (%)'] > df['1 Month Ago'])", ""]
cond3 = "& (df['1 Month Ago'] > df['3 Months Ago'])" # @param ["& (df['1 Month Ago'] > df['3 Months Ago'])", ""]
cond4 = "& (df['3 Months Ago'] > df['6 Months Ago'])" # @param ["& (df['3 Months Ago'] > df['6 Months Ago'])", ""]
cond5 = "& (df['6 Months Ago'] > df['9 Months Ago'])" # @param ["& (df['6 Months Ago'] > df['9 Months Ago'])", ""]
cond6 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond7 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond8 = "" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond9 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond10 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 11
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, ma, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nMA: {ma}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df_top_f2 = df.head(num_items)
display(df_top_f2)

print_column(df_top_f2, 'Name')
print_column(df_top_f2, 'Ticker')

Source: SPX+DJIA+NDX+RUI+SOX
Type: stocks
Period: 2y
MA: SMA
Date: 20240908


,Ticker,Sector,Industry,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,RS Rank (%),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
10,PPC,Consumer Defensive,Packaged Foods,30.20,29.38,25.27,15.74,12.03,-2.09,99.00,45.48,42.31,37.20,1.05,66.79,3.19,-1.09,75.197,14.19
20,K,Consumer Defensive,Packaged Foods,27.34,23.18,2.49,-5.44,-18.90,-20.26,98.00,80.19,68.79,61.36,0.69,46.17,2.51,8.31,37.479,31.91
42,HIW,Real Estate,REIT - Office,19.98,17.55,16.09,0.58,-3.06,-20.64,95.80,31.80,30.04,26.88,0.67,78.66,1.41,-4.84,7.858,22.38
58,WMT,Consumer Defensive,Discount Stores,17.74,13.96,7.60,5.97,-3.01,-6.72,94.21,76.96,71.92,65.31,1.09,10.55,1.92,-1.64,82.505,39.92
124,BK,Financial Services,Banks - Diversified,12.13,10.73,9.63,7.07,3.59,-1.01,87.61,68.00,64.62,59.75,0.93,34.98,4.20,-0.97,23.079,15.81
395,CMA,Financial Services,Banks - Regional,1.10,0.69,-3.69,-5.80,-6.68,-12.51,60.54,55.05,53.94,52.08,1.03,26.45,4.51,-3.19,24.498,12.17


PPC, K, HIW, WMT, BK, CMA


In [27]:
# @title Filter 3. RS Breakout {"run":"auto"}
base = 0 # @param ["0","30","60","100"] {"type":"raw"}
cond0 = "(df['RS (%)'] > base)" # @param ["(df['RS (%)'] > base)"]
cond1 = "& (df['1 Week Ago'] < base)" # @param ["& (df['1 Week Ago'] < base)",""]
cond2 = "& (df['1 Month Ago'] < base)" # @param ["& (df['1 Month Ago'] < base)",""]
cond3 = "& (df['3 Months Ago'] < base)" # @param ["& (df['3 Months Ago'] < base)",""]
cond4 = "& (df['6 Months Ago'] < base)" # @param ["& (df['6 Months Ago'] < base)",""]
cond5 = "& (df['9 Months Ago'] < base)" # @param ["& (df['9 Months Ago'] < base)",""]
cond6 = "& (df['Volume / VMA10'] > 3)" # @param ["& (df['Volume / VMA10'] > 1.5)", "& (df['Volume / VMA10'] > 2)", "& (df['Volume / VMA10'] > 3)", "& (df['Volume / VMA10'] > 4)", ""]
cond7 = "& (df['Price'] > df['MA30'])" # @param ["& (df['Price'] > df['MA10'])", "& (df['Price'] > df['MA30'])", ""]
cond8 = "" # @param ["& (df['MA10'] > df['MA30'])",""]
cond9 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond10 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond11 = "" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond12 = "" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond13 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", "& (df['Price'] < 500)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 14
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, ma, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nMA: {ma}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df_top_f3 = df.head(num_items)
display(df_top_f3)

print_column(df_top_f3, 'Name')
print_column(df_top_f3, 'Ticker')

Source: SPX+DJIA+NDX+RUI+SOX
Type: stocks
Period: 2y
MA: SMA
Date: 20240908


,Ticker,Sector,Industry,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,RS Rank (%),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE


### Step 3. Visualize Filtered Stocks

In [ ]:
# @title Plot 1. Mansfield Relative Strength Comparison {"run":"auto"}
source = "Filter 3. RS Breakout" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Major Global Stock Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1. Sorting': sorting_tickers,
    'Filter 2. Increasing RS': increasing_rs_tickers,
    'Filter 3. RS Breakout': rs_breakout_tickers,
    'Major Global Stock Indices': major_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
period_dropdown = create_period_dropdown()
interval_dropown = create_interval_dropdown()
cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'checkers'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
button = widgets.Button(description="Generate Plot")
out_msg, out_fig = widgets.Output(), widgets.Output()

menus = widgets.VBox([period_dropdown, interval_dropown, cmp_theme_dropdown])
display(checkbox_grid, menus, button, out_msg, out_fig)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

rs_cmp = {
    'mplfinance': mpl_mansfield.RelativeStrengthLines,
    'Plotly': ply_mansfield.RelativeStrengthLines,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_button_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        if rs_cmp is mpl_mansfield.RelativeStrengthLines:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        style=cmp_theme_dropdown.value)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        template=cmp_theme_dropdown.value)

button.on_click(on_button_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


GridBox(children=(Checkbox(value=True, description='建德工業', layout=Layout(width='auto'), style=DescriptionStyle…

Button(description='Generate Plot', style=ButtonStyle())

Output()

Output()

In [ ]:
# @title Plot 2. Mansfield Stock Chart {"run":"auto"}
source = "Filter 3. RS Breakout" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Major Global Stock Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import ipywidgets as widgets
from IPython.display import display, clear_output

symbols = {
    'Filter 1. Sorting': sorting_tickers,
    'Filter 2. Increasing RS': increasing_rs_tickers,
    'Filter 3. RS Breakout': rs_breakout_tickers,
    'Major Global Stock Indices': major_indices,
}[source]()

symbol_dropdown = widgets.Dropdown(
    options=symbols,
    value=symbols[0] if len(symbols) > 0 else None,
    description='Symbol:',
)

period_dropdown2 = create_period_dropdown('2y')
interval_dropown2 = create_interval_dropdown('1wk')
stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'simple_white'),
}[backend]

btn_plot_prc = widgets.Button(description="Plot Price/RS/Volume")
btn_plot_fin = widgets.Button(description="Plot Financials")
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
outputs = widgets.VBox()

display(
    symbol_dropdown,
    period_dropdown2, interval_dropown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

stock_chart = {
    'mplfinance': mpl_mansfield.StockChart,
    'Plotly': ply_mansfield.StockChart,
}[backend]

def on_plot_prc_click(b):
    selected = symbol_dropdown.value
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
            return
        interval = interval_dropown2.value
        period = period_dropdown2.value
        if stock_chart is mpl_mansfield.StockChart:
            stock_chart.plot(selected, interval=interval, period=period,
                             style=stock_theme_dropdown.value)
        else: # Plotly
            stock_chart.plot(selected, interval=interval, period=period,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = symbol_dropdown.value
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
            return
        if fin_chart is mpl_fin:
            fin_chart.plot(selected, style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected, template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


Dropdown(description='Symbol:', options=('迎輝', '建德工業', '同協', '柏文', '富堡'), value='迎輝')

Dropdown(description='Period:', index=1, options=('1y', '2y', '5y'), value='2y')

Dropdown(description='Interval:', index=1, options=('1d', '1wk'), value='1wk')

Dropdown(description='Stock Theme:', index=2, options=('default', 'classic', 'yahoo', 'charles', 'tradingview'…

VBox()